### Load and visualize dataset

In [1]:
import pandas as pd

df = pd.read_csv("data/selected_data.csv")

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
df.head(10)

,Politikbereich,Zweck
0,Verkehr,"Nord-Süd-Tangente; Linie 26/27, 2.2. Teil-BPU,A"
1,Bildung,Gedenken zu 30 Jahre Mauerfall
2,Stadtentwicklung,Lernen Na Logo - Bildungsnetzwerk Hellersdorfer Promenade
3,Jugend,Kinder- und Jugendambulanz
4,Jugend,Therapiebad
5,Integration,Stadtteilmütter in Neukölln
6,Jugend,Ausbau Familienzentrum
7,Jugend,Machbarkeitsstudie queeres Jugendzentrum
8,Jugend,Im Dorf der Hoffnung
9,Jugend,Projekttage VIELFÄLTIG FREIWILLIG


In [2]:
df["Zweck"].apply(lambda s: len(s.split())).sort_values(ascending = False).iloc[:10]

895     28
1084    27
832     26
401     26
841     24
233     24
96      23
621     21
225     20
191     20
Name: Zweck, dtype: int64

Observation: given the small size of the dataset (1138 examples), and the fact that the longest sentence has ~28 tokens, there is no reasonable reason for using bigrams or trigrams. Therefore, the model will be trained on lemmatized tokens and other text features.

### Data Augmentation (not used yet)

State of art NLP library (nlpaug) for text augmentation

Reference:
- https://github.com/makcedward/nlpaug
- https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb

In [3]:
import nlpaug.augmenter.word as naw

text = "Zusammen Fit für die Schule – Projekt zur Förderung der Sprachentwicklung von Vorschulkindern (FEIN Mittel)"

Insert word by contextual word embeddings (BERT, DistilBERT, RoBERTA or XLNet)

In [4]:
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-german-cased', action="insert")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Zusammen Fit für die Schule – Projekt zur Förderung der Sprachentwicklung von Vorschulkindern (FEIN Mittel)
Augmented Text:
Zusammen tragen Fit etwas für die Schule ist [UNK] das Projekt zur musikalischen Förderung der erfolgreichen Sprachentwicklung etwa von Vorschulkindern ( FEIN [UNK] Mittel )


Substitute word by contextual word embeddings (BERT, DistilBERT, RoBERTA or XLNet)

In [5]:
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-german-cased', action="substitute")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Zusammen Fit für die Schule – Projekt zur Förderung der Sprachentwicklung von Vorschulkindern (FEIN Mittel)
Augmented Text:
Am Fit für die Gruppe helfen Sie der Förderung der Betreuung von Vorschulkindern ( Eine AG )


### Text cleaning and normalization with Regex and Spacy

In [6]:
# !python -m spacy download de_core_news_lg

In [7]:
import re

def cleaner(text):
    # Remove mid slash and digits
    text = re.sub(r'-', ' ', text)
    text = re.sub(r'\d+', '', text)
    # Custom ones not supported by spacy
    text = re.sub(r'Abs\.', 'Absatz', text)
    text = re.sub(r'e\.V\.', 'eingetragener Verein', text)
    text = re.sub(r'co\.', 'Kompanie', text)
    text = re.sub(r'Co\.', 'Kompanie', text)
    text = re.sub(r'gem\.', 'gemäß', text)
    text = re.sub(r"'s", '', text)
    
    return text

df["Zweck"] = df["Zweck"].apply(cleaner)

In [8]:
import spacy

nlp = spacy.load("de_core_news_lg")

nlp.remove_pipe("ner")
nlp.remove_pipe("parser")
nlp.remove_pipe("attribute_ruler")
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x14787f29220>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x14787e279a0>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x14787e27a00>),
 ('lemmatizer', <spacy.pipeline.lemmatizer.Lemmatizer at 0x14787fb3200>)]

In [9]:
def normalize(text):
    doc = nlp(text)
    output = []
    for token in doc:
        if not token.is_punct and not token.is_stop and not token.is_space:
            output.append(token.lemma_.lower())
    return " ".join(output)

df["Zweck"] = df["Zweck"].apply(normalize)
df.head(10)

,Politikbereich,Zweck
0,Verkehr,nord süd tangente linie bpu
1,Bildung,gedenken mauerfall
2,Stadtentwicklung,lernen logo bildungsnetzwerk hellersdorfer promenade
3,Jugend,kind jugendambulanz
4,Jugend,therapiebad
5,Integration,stadtteilmütter neukölln
6,Jugend,ausbau familienzentrum
7,Jugend,machbarkeitsstudie queeres jugendzentrum
8,Jugend,dorf hoffnung
9,Jugend,projekttage vielfältig freiwillig


### Save the preprocessed dataset

In [10]:
df.to_csv("data/preprocessed_data.csv", index = False)